## <center> Régression Linéaire et Moindres Carrés</center>
#### <center>Michal Urdanivia (UGA)</center>
#### <center> michal.wong-urdanivia@univ-grenoble-alpes.fr </center>

#### Références

En ce qui concerne l'utilisation de Python pour le travail empirique nous ne pourrons pas aller au-delà de ce que nous utilisons dans le cours. Pour vous former sur ce langage et son utilisation dans le travail empirique vous pouvez consulter:

* Le site [QuantEcon](https://quantecon.org/), en particulier les "lectures" en ["Data Science"](https://datascience.quantecon.org/)

* Le site que [Jake VanderPlas](https://jakevdp.github.io/pages/about.html) dédie à son [ouvrage sur Python en Data Science](https://jakevdp.github.io/PythonDataScienceHandbook/).



## <center>  Données d'application </center>

Nous allons utiliser des données qui sont disponible sur le site que [Bruce Hansen](https://www.ssc.wisc.edu/~bhansen/) dédie à son cours d'[économétrie](https://www.ssc.wisc.edu/~bhansen/econometrics/). Plus précisément, nous allons utiliser des données extraites du **Current Population Survey**(CPS) de 2009. Une description du fichier est [ici](https://www.ssc.wisc.edu/~bhansen/econometrics/cps09mar_description.pdf)



#### Lecture des données

Pour travailler(e.g., lire/charger les données, sélectionner des variables, etc) sur les données nous allons utiliser la bibliothèque [pandas](https://fr.wikipedia.org/wiki/Pandas) dont vous n'avez pas besoin de lire toute la documentation pour l'utiliser(quelqu'un a t-il déjà lu les quelques 3000 pages correspondantes à la doc en pdf? ou au moins 50 %?)

In [1]:
# Import/appel de la bibliothèque pandas et autres qui seront utilisées 
# Remarques: 
# - les écritures après le "#" sont des commentaires non considérés comme du code à executer.
# - pd, np, etc ci-après sont des abréviations que nous donnons aux bibliothèques correspondates
# (elle sont courantes comme vous pourrez le constater en regardant un peu sur le web)


import pandas as pd   
import numpy as np
import statsmodels.api as sm
from sklearn import linear_model

In [2]:
# Lecture des données.
# On utilise la fonction "read_stata" dans pandas pour lire le fichier au format stata(".dta") disponible 
# sur le site de Bruce Hansen. Vous pouvez aussi le télécharger sur votre poste et ensuite le lire.
# Nous l'appellons cps_df(pour cps data frame)

cps_df = pd.read_stata("https://www.ssc.wisc.edu/~bhansen/econometrics/cps09mar.dta")
cps_df.info()   # Affichage d'informations.


<class 'pandas.core.frame.DataFrame'>
Int64Index: 50742 entries, 0 to 50741
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        50742 non-null  float64
 1   female     50742 non-null  float64
 2   hisp       50742 non-null  float64
 3   education  50742 non-null  float64
 4   earnings   50742 non-null  float64
 5   hours      50742 non-null  float64
 6   week       50742 non-null  float64
 7   union      50742 non-null  float64
 8   uncov      50742 non-null  float64
 9   region     50742 non-null  float64
 10  race       50742 non-null  float64
 11  marital    50742 non-null  float64
dtypes: float64(12)
memory usage: 5.0 MB


In [3]:
# Affichage des premières lignes(5 par défaut) et de statistiques descriptives de base(moyennes, écart-types, etc)
print(cps_df.head())
cps_df.describe()

    age  female  hisp  education  earnings  hours  week  union  uncov  region  \
0  52.0     0.0   0.0       12.0  146000.0   45.0  52.0    0.0    0.0     1.0   
1  38.0     0.0   0.0       18.0   50000.0   45.0  52.0    0.0    0.0     1.0   
2  38.0     0.0   0.0       14.0   32000.0   40.0  51.0    0.0    0.0     1.0   
3  41.0     1.0   0.0       13.0   47000.0   40.0  52.0    0.0    0.0     1.0   
4  42.0     0.0   0.0       13.0  161525.0   50.0  52.0    1.0    0.0     1.0   

   race  marital  
0   1.0      1.0  
1   1.0      1.0  
2   1.0      1.0  
3   1.0      1.0  
4   1.0      1.0  


,age,female,hisp,education,earnings,hours,week,union,uncov,region,race,marital
count,50742.000000,50742.000000,50742.000000,50742.000000,50742.000000,50742.000000,50742.000000,50742.000000,50742.000000,50742.000000,50742.000000,50742.000000
mean,42.131725,0.425722,0.148792,13.924619,55091.530685,43.827244,51.879272,0.021521,0.002207,2.635627,1.433507,2.763174
std,11.487620,0.494457,0.355887,2.744447,52222.071166,7.704467,0.598646,0.145113,0.046930,1.060051,1.317430,2.503158
min,15.000000,0.000000,0.000000,0.000000,1.000000,36.000000,48.000000,0.000000,0.000000,1.000000,1.000000,1.000000
25%,33.000000,0.000000,0.000000,12.000000,28000.000000,40.000000,52.000000,0.000000,0.000000,2.000000,1.000000,1.000000
50%,42.000000,0.000000,0.000000,13.000000,42000.000000,40.000000,52.000000,0.000000,0.000000,3.000000,1.000000,1.000000
75%,51.000000,1.000000,0.000000,16.000000,65000.000000,45.000000,52.000000,0.000000,0.000000,4.000000,1.000000,5.000000
max,85.000000,1.000000,1.000000,20.000000,561087.000000,99.000000,52.000000,1.000000,1.000000,4.000000,21.000000,7.000000


## <center>  Régression linéaire et estimation par MCO </center>

Le modèle que nous allons estimer suppose que,

$$
\begin{align*}
Y &= X^\top \beta + U, \tag{1}
\end{align*}
$$

où $ Y\in \mathbb{R} $, $ X\in\mathbb{R}^K $, $ U\in \mathbb{R} $, sont de variables aléatoires: $ Y $ est la variables dépendante, 
$ X $ sont $ K $ régresseurs, $ U $ est l'erreur du modèle. $ Y $ et $ X $ sont observables tandis que $ U $ représente tout ce qui peut faire varier $ Y $ pour des valeurs données de $ X $. $ \beta \in \mathbb{R}^K $ sont les paramètres inconnus du modèle qu'on se propose d'estimer car ils permettent de mesurer la relation entre les éléments de $ X $ et $ Y $ sans pour autant que cette relation si elle existe puisse avoir une interprétation causale ou en termers de modèle de régression. 

En effet, pour que $ \beta $ dans (1) soit un vecteur de paramètres d'un modèle de régression linéaire, à savoir d'un modèle tel que
 $ \operatorname{E}(Y|X) = X^\top \beta $, on doit supposer que $ \operatorname{E}(U|X) = 0 $. En particulier, c'est sous cette condition(ainsi que d'autres vues en cours) que l'estimateur des MCO de $ \beta $ sera sans biais et convergent. Notons néanmoins que la convergence peut être obtenue  en replaçant  la condition d'**exogénéité forte** $ \operatorname{E}(U|X) = 0$ par celle **exogénéité faible** $ \operatorname{E}(XU) = 0$(mais dans ce cas l'estimateur des MCO n'est plus sans biais).

Pour estimer $ \beta $ on s'appuie sur des données de $ (Y, X) $ qu'on suppose i.i.d. $ \{(Y_i, X_i)\}_{i=1}^n $ et ainsi,

$$
\begin{align*}
Y_i &= X_i^\top \beta + U_i, \ i=1, \ldots, n. \tag{2}
\end{align*}
$$

L'estimateur des MCO de $ \beta $ est alors défini par,

$$
\begin{align*}
\hat{\beta}_n^{MCO} &=  \argmin_{b\in \mathbb{R}^K} n^{-1}\sum_{i=1}^n(Y_i - X_i^\top b)^2 \tag{3}
\end{align*}
$$

et les conditions du premier ordre, 

$$
\begin{align*}
n^{-1}\sum_{i=1}^n X_i(Y_i - X_i^\top b) &=0,
\end{align*}
$$
permettent de calculer $ \hat{\beta}_n^{MCO} $ dès lors que $ n^{-1}\sum_{i=1}^n X_iX_i^\top $ est de plein rang ce qui revient à l'absence de multicolinéarité entre régrésseurs dans l'échantillon. On obtient alors,

$$
\begin{align*}
\hat{\beta}_n^{MCO} &= \left( n^{-1}\sum_{i=1}^n X_iX_i^\top \right)^{-1} n^{-1}\sum_{i=1}^n X_iY_i \tag{4}
\end{align*}
$$



On peut alors définir $\hat{U}_i = Y_i -X_i^\top\hat{\beta}_n^{MCO}$ avec $\hat{\beta}_n^{MCO}$ donné par (4) et écrire la décomposition

$$
\begin{align*}
Y_i &= X_i^\top\hat{\beta}_n^{MCO} + \hat{U}_i, \ \ n^{-1}\sum_{i=1}^n X_i\hat{U}_i = 0. \tag{5}
\end{align*}
$$


Notons que le facteur "$ n^{-1} $"  dans (3) souligne que la que la fonction objectif pour obtenir l'estimateur des MCO est la contrepartie empirique de celle dans le problème,

$$
\begin{align*}
\beta &=  \argmin_{b\in \mathbb{R}^K}\operatorname{E}\left((Y_i - X_i^\top b)^2\right) \tag{6}
\end{align*}
$$

qui a pour conditions du premier ordre,

$$
\begin{align*}
\operatorname{E}\left(X_i(Y_i - X_i^\top b) \right) &=0,
\end{align*}
$$

et à condition que $ \operatorname{E}(X_iX_i^\top) $ soit de plein rang ce qui revient à l'absence de multicolinéarité entre régrésseurs dans la population théorique, on obtient,

$$
\begin{align*}
\beta &= \left( \operatorname{E}(X_iX_i^\top) \right)^{-1} \operatorname{E}(X_iY_i) \tag{7}
\end{align*}
$$

On peut alors définir $U_i = Y_i- X_i^\top\beta$ avec $\beta$ donné par (7) et écrire la décomposition

$$
\begin{align*}
Y_i &= X_i^\top\beta + U_i, \ \  \operatorname{E}(X_iU_i) = 0. \tag{8}
\end{align*}
$$


Donc $ \hat{\beta}_n^{MCO} $ peut être calculé indépendamment des conditions faites sur la relation entre $ X $ et $ U $ dans (1)(à condition de les éléments de $ X_i $ ne soient pas colinéaires dans l'échantillon). L'importance de ces conditions tient au propriétés qu'elles permettent d'établir(e.g., convergence, absence de biais) et aux interprétations(e.g., estimation des paramètres d'un modèle de régression). 
En particulier:

- Avec la condition $ \operatorname{E}(U_i|X_i) = 0 $, l'estimateur des MCO est celui des paramètres d'un modèle de régression linéaire, à savoir tel que (2) et $ \operatorname{E}(Y_i|X_i) = X_i^\top\beta $. Cet estimateur est convergent, et sans biais.

- Avec la condition $ \operatorname{E}(X_iU_i) = 0 $ l'estimateur des MCO est celui des paramètres d'un modèle tel que (8). C'est ce qu'on appelle un estimateur les paramètres de la projection de $ Y_i $ sur $ X_i $.  Cet estimateur est seulement convergent.

#### Application.

Nous allons considérer un modèle où la variable dépendante est un log du salaire, et les régresseurs incluent une constante, le niveau d'études, une mesure de l'expérience en années, son carré, une indicatrice du sexe, et une indicatrice d'être noir.  Soit,

- $ Y $: $ lwage $,
- $ X $: $ (1, educ, exper, expersq, female, black)$.

Nous allons utiliser un échantillon qui correspond aux personnes qui se définissent comme blanches, ou noires(remarque: la collecte de données américaines permet de recueillir des informations quant à l'appartenance à des groupes ethniques préalablement définis). Certaines des variables utilisés doivent d'abord être calculées.


In [4]:
# Échantillon

cps_df2 = cps_df[(cps_df.race == 1.0) | (cps_df.race == 2.0)]

# Variables

cps_df2 = cps_df2.assign(exper = cps_df2.age - cps_df2.education - 6) # Expérience
cps_df2 = cps_df2.assign(expersq = cps_df2.exper**2/100) # Expérience au carré
cps_df2 = cps_df2.assign(lwage = np.log(cps_df2.earnings / ( cps_df2.hours * cps_df2.week))) # revenu horaire
cps_df2 = pd.get_dummies(data = cps_df2, columns= ['race']) # indicatrice d'appartenance ethnique
cps_df2 = cps_df2.rename(columns={"race_1.0": "white", "race_2.0": "black"}) # on les renomme 
print(cps_df2.shape)
cps_df2.describe()
#cps_df2[['exper', 'age', 'education', 'expersq', 'lwage', 'earnings', 'week', 'hours']].head()


(46411, 16)


,age,female,hisp,education,earnings,hours,week,union,uncov,region,marital,exper,expersq,lwage,white,black
count,46411.000000,46411.000000,46411.000000,46411.000000,46411.000000,46411.000000,46411.000000,46411.000000,46411.000000,46411.000000,46411.000000,46411.000000,46411.000000,46411.000000,46411.000000,46411.000000
mean,42.213915,0.423477,0.154468,13.882269,55082.729181,43.879964,51.879554,0.021934,0.002262,2.597789,2.754584,22.331646,6.337939,2.945706,0.889358,0.110642
std,11.468616,0.494115,0.361400,2.713667,52324.915589,7.701222,0.596815,0.146471,0.047511,1.047513,2.497897,11.623014,5.635276,0.673137,0.313692,0.313692
min,15.000000,0.000000,0.000000,0.000000,1.000000,36.000000,48.000000,0.000000,0.000000,1.000000,1.000000,-4.000000,0.000000,-7.863267,0.000000,0.000000
25%,33.000000,0.000000,0.000000,12.000000,28000.000000,40.000000,52.000000,0.000000,0.000000,2.000000,1.000000,13.000000,1.690000,2.560096,1.000000,0.000000
50%,42.000000,0.000000,0.000000,13.000000,42000.000000,40.000000,52.000000,0.000000,0.000000,3.000000,1.000000,22.000000,4.840000,2.956512,1.000000,0.000000
75%,51.000000,1.000000,0.000000,16.000000,65000.000000,45.000000,52.000000,0.000000,0.000000,3.000000,5.000000,31.000000,9.610000,3.354542,1.000000,0.000000
max,85.000000,1.000000,1.000000,20.000000,561087.000000,99.000000,52.000000,1.000000,1.000000,4.000000,7.000000,75.000000,56.250000,5.583706,1.000000,1.000000


#### Calcul avec modules existants.

En général pour une tâche donnée, il est recommandé d'utiliser des modules/bibliothèques/fonctions déjà existantes plutôt que de chercher à les reprogrammer de zéro(bien que cet exercice soit très instructif) car ces fonctions on souvent été déjà largement testées, et surtout cela permet 
d'envisage des développements/améliorations sur des bases établies.

Par exemple, pour calculer un estimateur des MCO il existe énormément de modules disponible largement employés par la communauté. Nous allons en considérer deux associés à deux bibliothèques parmi les plus populaires:

- [statsmodels](https://www.statsmodels.org/stable/index.html): orienté statistique au sens général.

- [scikit-learn](https://scikit-learn.org/stable/): davantage apprentissage.

**statsmodels**

Une syntaxe de base est la suivante:

In [47]:
model = sm.OLS(dep_var, sm.add_constant(reg_var))
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.280
Model:                            OLS   Adj. R-squared:                  0.280
Method:                 Least Squares   F-statistic:                     3602.
Date:                Mon, 07 Feb 2022   Prob (F-statistic):               0.00
Time:                        19:25:05   Log-Likelihood:                -39873.
No. Observations:               46411   AIC:                         7.976e+04
Df Residuals:                   46405   BIC:                         7.981e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0208      0.017     60.495      0.0

Pour avoir des écart-types estimés robustes à l'hétéroscédasticité vous pouvez faire:

In [45]:
results2 = model.fit(cov_type='HC0')
print(results2.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.280
Model:                            OLS   Adj. R-squared:                  0.280
Method:                 Least Squares   F-statistic:                     2869.
Date:                Mon, 07 Feb 2022   Prob (F-statistic):               0.00
Time:                        19:10:09   Log-Likelihood:                -39873.
No. Observations:               46411   AIC:                         7.976e+04
Df Residuals:                   46405   BIC:                         7.981e+04
Df Model:                           5                                         
Covariance Type:                  HC0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0208      0.019     55.132      0.0

**Remarques**

- Comme pour le type d'écart-types plusieurs options sous forme d'arguments de la fonction existent. Pour plus de détails vous pouvez 
consulter la [documentation](https://www.statsmodels.org/dev/regression.html) et ou faire des rechercher(sur le web) en fonction de vos besoins.

- L'option HC0 est une parmi d'autres disponibles pour les écart-types robustes à l'hétéroscédasticité, les autres étant 'HC1', 'HC2', 'HC3'. Pour de détails vous pouvez regarder la section 4.13 du [cours en pdf](https://www.ssc.wisc.edu/~bhansen/econometrics/Econometrics.pdf) de Bruce Hansen.